In [1]:
import * as duckdb from 'https://cdn.jsdelivr.net/npm/@duckdb/duckdb-wasm@1.29.0/+esm';

const JSDELIVR_BUNDLES = duckdb.getJsDelivrBundles();

// Select a bundle based on browser checks
const bundle = await duckdb.selectBundle(JSDELIVR_BUNDLES);

const worker_url = URL.createObjectURL(
  new Blob([`importScripts("${bundle.mainWorker}");`], { type: 'text/javascript' })
);


const worker = new Worker(worker_url);
const logger = new duckdb.ConsoleLogger();
const db = new duckdb.AsyncDuckDB(logger, worker);
await db.instantiate(bundle.mainModule, bundle.pthreadWorker);
URL.revokeObjectURL(worker_url);

In [2]:
let conn = await db.connect()

const result = await conn.query("SELECT 42 as col")

pprint(result.toArray().map(row => row.toJSON()))


[object Object]
[
  {
    "col": 42
  }
]


In [7]:
const url = 'https://raw.githubusercontent.com/plotly/datasets/master/2015_flights.parquet';

const response = await fetch(url);
if (!response.ok){
    throw Error("Failed to fetch")
}

const buffer = await response.arrayBuffer();
const uint8Array = new Uint8Array(buffer);
await db.registerFileBuffer('2015_flights.parquet', uint8Array);


const data = await conn.query(`
SELECT * FROM read_parquet('2015_flights.parquet') LIMIT 10;
`
)

const rows = data.toArray().map(row => {
  const jsonRow = row.toJSON();
  for (const key in jsonRow) {
    if (typeof jsonRow[key] === 'bigint') {
      jsonRow[key] = jsonRow[key].toString();
    }
  }
  return jsonRow;
});

// ijs.display.json(rows);
console.log(data.toString())

[object Object]
[
  {"DEPARTURE_DELAY": -11, "ARRIVAL_DELAY": -22, "DISTANCE": 1448, "SCHEDULED_DEPARTURE": 0.08333333333333333},
  {"DEPARTURE_DELAY": -8, "ARRIVAL_DELAY": -9, "DISTANCE": 2330, "SCHEDULED_DEPARTURE": 0.16666666666666666},
  {"DEPARTURE_DELAY": -2, "ARRIVAL_DELAY": 5, "DISTANCE": 2296, "SCHEDULED_DEPARTURE": 0.3333333333333333},
  {"DEPARTURE_DELAY": -5, "ARRIVAL_DELAY": -9, "DISTANCE": 2342, "SCHEDULED_DEPARTURE": 0.3333333333333333},
  {"DEPARTURE_DELAY": -1, "ARRIVAL_DELAY": -21, "DISTANCE": 1448, "SCHEDULED_DEPARTURE": 0.4166666666666667},
  {"DEPARTURE_DELAY": -5, "ARRIVAL_DELAY": 8, "DISTANCE": 1589, "SCHEDULED_DEPARTURE": 0.4166666666666667},
  {"DEPARTURE_DELAY": -6, "ARRIVAL_DELAY": -17, "DISTANCE": 1299, "SCHEDULED_DEPARTURE": 0.4166666666666667},
  {"DEPARTURE_DELAY": 14, "ARRIVAL_DELAY": -10, "DISTANCE": 2125, "SCHEDULED_DEPARTURE": 0.5},
  {"DEPARTURE_DELAY": -11, "ARRIVAL_DELAY": -13, "DISTANCE": 1464, "SCHEDULED_DEPARTURE": 0.5},
  {"DEPARTURE_DELAY": 3,